In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/sensor 3 data.csv", delimiter=";")
df

In [ ]:
df2 = pd.read_csv("/content/pump 3 data.csv", delimiter=";")
df2

In [ ]:
df3 = df.drop(columns=['New Device Time', 'Index', 'BG Source', 'BG Reading (mg/dL)', 'Linked BG Meter ID', 'Basal Rate (U/h)', 'Temp Basal Amount', 'Temp Basal Type', 'Temp Basal Duration (h:mm:ss)', 'Bolus Type', 'Bolus Volume Selected (U)', 'Bolus Duration (h:mm:ss)', 'Prime Type', 'Prime Volume Delivered (U)', 'Alarm', 'Suspend', 'Rewind', 'BWZ Estimate (U)', 'BWZ Target High BG (mg/dL)', 'BWZ Target Low BG (mg/dL)', 'BWZ Insulin Sensitivity (mg/dL/U)', 'BWZ BG/SG Input (mg/dL)', 'BWZ Correction Estimate (U)', 'BWZ Food Estimate (U)', 'BWZ Active Insulin (U)', 'BWZ Status', 'Sensor Calibration BG (mg/dL)', 'Event Marker', 'Bolus Number', 'Bolus Cancellation Reason', 'BWZ Unabsorbed Insulin Total (U)', 'Final Bolus Estimate', 'Scroll Step Size', 'Insulin Action Curve Time', 'Sensor Calibration Rejected Reason', 'Preset Bolus', 'Bolus Source', 'BLE Network Device', 'Device Update Event', 'Network Device Associated Reason', 'Network Device Disassociated Reason', 'Network Device Disconnected Reason', 'Sensor Exception', 'Preset Temp Basal Name'])
df3

In [ ]:
df4 = df2.drop(columns=['New Device Time', 'Index', 'BG Source', 'BG Reading (mg/dL)', 'Linked BG Meter ID', 'Basal Rate (U/h)', 'Temp Basal Amount', 'Temp Basal Type', 'Temp Basal Duration (h:mm:ss)', 'Bolus Type', 'Bolus Volume Selected (U)', 'Bolus Duration (h:mm:ss)', 'Prime Type', 'Prime Volume Delivered (U)', 'Alarm', 'Suspend', 'Rewind', 'BWZ Estimate (U)', 'BWZ Target High BG (mg/dL)', 'BWZ Target Low BG (mg/dL)', 'BWZ Insulin Sensitivity (mg/dL/U)', 'BWZ BG/SG Input (mg/dL)', 'BWZ Correction Estimate (U)', 'BWZ Food Estimate (U)', 'BWZ Active Insulin (U)', 'BWZ Status', 'Sensor Calibration BG (mg/dL)', 'Event Marker', 'Bolus Number', 'Bolus Cancellation Reason', 'BWZ Unabsorbed Insulin Total (U)', 'Final Bolus Estimate', 'Scroll Step Size', 'Insulin Action Curve Time', 'Sensor Calibration Rejected Reason', 'Preset Bolus', 'Bolus Source', 'BLE Network Device', 'Device Update Event', 'Network Device Associated Reason', 'Network Device Disassociated Reason', 'Network Device Disconnected Reason', 'Sensor Exception', 'Preset Temp Basal Name'])
df4= df4.iloc[:20934]
df4

In [ ]:
df3['datetime'] = pd.to_datetime(df3['Date'] + ' ' + df3['Time'])


df3.drop(columns=['Date', 'Time'], inplace=True)
df3

In [ ]:
df4['datetime'] = pd.to_datetime(df4['Date'] + ' ' + df4['Time'])

df4.drop(columns=['Date', 'Time'], inplace=True)
df4

In [ ]:
df3 = df3.sort_values(by='datetime')
df4 = df4.sort_values(by='datetime')
df3_indexed = df3.set_index('datetime')
df4_indexed = df4.set_index('datetime')
df5 = df3_indexed.join(df4_indexed, how='outer', lsuffix='_df3', rsuffix='_df4')
df5

In [ ]:
df5 = df5.drop(columns=['Bolus Volume Delivered (U)_df3', 'BWZ Carb Ratio (g/U)_df3', 'BWZ Carb Input (grams)_df3', 'Sensor Glucose (mg/dL)_df4', 'ISIG Value_df4'])
df5

In [ ]:
df5["Sensor Glucose (mg/dL)_df3"] = df5["Sensor Glucose (mg/dL)_df3"].fillna(method="ffill")
df5["ISIG Value_df3"] = df5["ISIG Value_df3"].fillna(method="ffill")
df5["BWZ Carb Ratio (g/U)_df4"] = df5["BWZ Carb Ratio (g/U)_df4"].fillna(method="ffill")


In [ ]:
df5

In [ ]:
df5["BWZ Carb Ratio (g/U)_df4"] = df5["BWZ Carb Ratio (g/U)_df4"].fillna(method="bfill")
df5

In [ ]:
df5["Bolus Volume Delivered (U)_df4"] = df5["Bolus Volume Delivered (U)_df4"].fillna(0)
df5["BWZ Carb Input (grams)_df4"] = df5["BWZ Carb Input (grams)_df4"].fillna(0)
df5

In [ ]:
df5 = df5.iloc[1:]
df5

In [ ]:
df5.describe()


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plot_acf(df5['Sensor Glucose (mg/dL)_df3'], lags=30, title='Autocorrelation of glucose')
plt.show()


plt.figure(figsize=(12, 6))
plot_pacf(df5['Sensor Glucose (mg/dL)_df3'], lags=30, title='Partial Autocorrelation of glucose')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
correlation_matrix = df5.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


df5['time_diff'] = df5.index.to_series().diff().dt.total_seconds()


df5['cumulative_time'] = df5['time_diff'].cumsum()


df5['target'] = df5['Sensor Glucose (mg/dL)_df3'].shift(-1)
for i in range(1, len(df5)):
    if df5['cumulative_time'].iloc[i] >= 3600:
        df5['target'].iloc[i-1] = df5['Sensor Glucose (mg/dL)_df3'].iloc[i]
        df5['cumulative_time'] = df5['cumulative_time'] - 3600


df5 = df5.dropna()



for lag in range(1, 31):
    df5.loc[:, f'lag_{lag}'] = df5['Sensor Glucose (mg/dL)_df3'].shift(lag)


df5 = df5.dropna()

additional_features = ['ISIG Value_df3', 'Bolus Volume Delivered (U)_df4', 'BWZ Carb Ratio (g/U)_df4', 'BWZ Carb Input (grams)_df4']


X = df5[[f'lag_{i}' for i in range(1, 31)] + additional_features]
y = df5['target']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)


train_pred = model.predict(X_train)
test_pred = model.predict(X_test)


train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")


plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="True Values", color="blue")
plt.plot(test_pred, label="Predictions", color="red", alpha=0.7)
plt.title("True Values vs Predictions")
plt.xlabel("Time")
plt.ylabel("Sensor Glucose (mg/dL)")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error



for lag in range(1, 31):
    df5.loc[:, f'lag_{lag}'] = df5['Sensor Glucose (mg/dL)_df3'].shift(lag)


df5 = df5.dropna()


additional_features = ['ISIG Value_df3', 'Bolus Volume Delivered (U)_df4', 'BWZ Carb Ratio (g/U)_df4', 'BWZ Carb Input (grams)_df4']


X = df5[[f'lag_{i}' for i in range(1, 31)] + additional_features]
y = df5['Sensor Glucose (mg/dL)_df3']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


num_features = X.shape[1]
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, num_features))


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))


model.compile(optimizer='adam', loss='mean_squared_error')


history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', color='blue')
plt.plot(y_test_pred, label='Predicted', color='red', alpha=0.7)
plt.title('Sensor Glucose Prediction with LSTM')
plt.xlabel('Samples')
plt.ylabel('Sensor Glucose (mg/dL)')
plt.legend()
plt.show()
